# Exercise 25

For each input line, substitute the 'age' attribute with the corresponding range ('rangeage').


In [ ]:
# As usual, clear the destination folder to prevent overwriting
!hdfs dfs -rm -r /user/s315054/ex_results/res_out_Ex25

In [1]:
inPath = "/data/students/bigdata_internet/exercises/Ex25/data/profiles.csv"
outPath = "/user/s315054/ex_results/res_out_Ex25"

In [2]:
in_DF = spark.read.load(inPath, format='csv', header=True, inferSchema=True)

in_DF.show(5)

+-----+-------+---+
| name|surname|age|
+-----+-------+---+
|Paolo|  Garza| 42|
| Luca| Boccia| 41|
|Maura|Bianchi| 16|
+-----+-------+---+



In [10]:
spark.udf.register("getRangeAge", lambda age: "[" + str(int(age/10)*10) + "-" + str(int(age/10)*10 + 9) + "]")

<function __main__.<lambda>(age)>

In [11]:
out_DF = in_DF.selectExpr('name', 'surname', 'getRangeAge(age) AS rangeage')

out_DF.show(5)

+-----+-------+--------+
| name|surname|rangeage|
+-----+-------+--------+
|Paolo|  Garza| [40-49]|
| Luca| Boccia| [40-49]|
|Maura|Bianchi| [10-19]|
+-----+-------+--------+



In [12]:
out_DF.write.csv(outPath, header=True)

In [17]:
try_DF = in_DF.selectExpr('name', 'age', 'age*4 + 2 AS new_age')

try_DF.show(5)

+-----+---+-------+
| name|age|new_age|
+-----+---+-------+
|Paolo| 42|    170|
| Luca| 41|    166|
|Maura| 16|     66|
+-----+---+-------+



In [18]:
DF = try_DF.selectExpr('name', 'age/new_age')

DF.show(5)

+-----+-------------------+
| name|    (age / new_age)|
+-----+-------------------+
|Paolo|0.24705882352941178|
| Luca| 0.2469879518072289|
|Maura|0.24242424242424243|
+-----+-------------------+



## Using SQL queries

In [15]:
!hdfs dfs -rm -r /user/s315054/ex_results/res_out_Ex25

22/11/20 14:47:06 INFO fs.TrashPolicyDefault: Moved: 'hdfs://BigDataHA/user/s315054/ex_results/res_out_Ex25' to trash at: hdfs://BigDataHA/user/s315054/.Trash/Current/user/s315054/ex_results/res_out_Ex25


In [13]:
in_DF.createOrReplaceTempView('profiles')

In [14]:
out_DF_2 = spark.sql("SELECT name, surname, getRangeAge(age) AS rangeage FROM profiles")

out_DF_2.show(5)

+-----+-------+--------+
| name|surname|rangeage|
+-----+-------+--------+
|Paolo|  Garza| [40-49]|
| Luca| Boccia| [40-49]|
|Maura|Bianchi| [10-19]|
+-----+-------+--------+



In [16]:
out_DF_2.write.csv(outPath, header=True)